In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time, json, os, csv, requests

from scipy.stats import linregress
from citipy import citipy
from pprint import pprint

from function.helper import clean_house_data, clean_school_data, clean_crime_data

# Hi Markers, please use your onw API keys to run the script, thank you!
from api_keys import weather_api_key, g_key

# Set source file path
house_data_path = "data_source/Melbourne_housing_FULL.csv"
school_data_path = "data_source/dv309_schoollocations2021.csv"
crime_rate_path = "data_source/Data_Tables_LGA_Criminal_Incidents_Year_Ending_March_2021.xlsx"

# Set output file path
house_file = "data_source/cleaned_source_data/house_cleaned.csv"
school_file = "data_source/cleaned_source_data/school_cleaned.csv"
crime_file = "data_source/cleaned_source_data/crime_cleaned.csv"

In [2]:
house_data = pd.read_csv(house_data_path)
house_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [3]:
column_to_int_list = ["Postcode",
                      "Bathroom",
                      "Car",
                      "Landsize",
                      "BuildingArea",
                      "YearBuilt",
                      "Propertycount",
                      "Rooms"]

cleaned_house_data = clean_house_data(house_data, column_to_int_list)

cleaned_house_data.to_csv(house_file, index = False, encoding = "utf-8")

cleaned_house_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
1,Abbotsford,85 Turner St,2,h,"1,480,000",S,Biggin,3/12/2016,2.5,3067,1,1,202,0,0,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019
2,Abbotsford,25 Bloomburg St,2,h,"1,035,000",S,Biggin,4/02/2016,2.5,3067,1,0,156,79,1900,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019
4,Abbotsford,5 Charles St,3,h,"1,465,000",SP,Biggin,4/03/2017,2.5,3067,2,0,134,150,1900,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019
5,Abbotsford,40 Federation La,3,h,"850,000",PI,Biggin,4/03/2017,2.5,3067,2,1,94,0,0,Yarra City Council,-37.7969,144.9969,Northern Metropolitan,4019
6,Abbotsford,55a Park St,4,h,"1,600,000",VB,Nelson,4/06/2016,2.5,3067,1,2,120,142,2014,Yarra City Council,-37.8072,144.9941,Northern Metropolitan,4019


In [4]:
school_data = pd.read_csv(school_data_path, encoding = "cp1252")
school_data.head()

,Education_Sector,Entity_Type,SCHOOL_NO,School_Name,School_Type,School_Status,Address_Line_1,Address_Line_2,Address_Town,Address_State,...,Postal_Address_Line_1,Postal_Address_Line_2,Postal_Town,Postal_State,Postal_Postcode,Full_Phone_No,LGA_ID,LGA_Name,X,Y
0,Government,1,1,Alberton Primary School,Primary,O,21 Thomson Street,NaN,Alberton,VIC,...,21 Thomson Street,NaN,ALBERTON,VIC,3971,03 5183 2412,681,Wellington (S),146.666601,-38.617713
1,Government,1,3,Allansford and District Primary School,Primary,O,Frank Street,NaN,Allansford,VIC,...,Frank Street,NaN,ALLANSFORD,VIC,3277,03 5565 1382,673,Warrnambool (C),142.590393,-38.386281
2,Government,1,4,Avoca Primary School,Primary,O,118 Barnett Street,NaN,Avoca,VIC,...,P O Box 12,NaN,AVOCA,VIC,3467,03 5465 3176,599,Pyrenees (S),143.475649,-37.084502
3,Government,1,8,Avenel Primary School,Primary,O,40 Anderson Street,NaN,Avenel,VIC,...,40 Anderson Street,NaN,AVENEL,VIC,3664,03 5796 2264,643,Strathbogie (S),145.234722,-36.901368
4,Government,1,12,Warrandyte Primary School,Primary,O,5-11 Forbes Street,NaN,Warrandyte,VIC,...,5-11 Forbes Street,NaN,WARRANDYTE,VIC,3113,03 9844 3537,421,Manningham (C),145.213980,-37.742675


In [5]:
cleaned_school_data = clean_school_data(school_data)

cleaned_school_data.to_csv(school_file, index = False, encoding = "utf-8")

cleaned_school_data.head()

,Education_Sector,Entity_Type,School_Name,School_Type,School_Status,Address,Suburb,State,Postcode,Full_Phone_No,LGA_ID,LGA_Name,Lng,Lat
0,Government,1,Alberton Primary School,Primary,O,21 Thomson Street,Alberton,VIC,3971,03 5183 2412,681,Wellington (S),146.666601,-38.617713
1,Government,1,Allansford and District Primary School,Primary,O,Frank Street,Allansford,VIC,3277,03 5565 1382,673,Warrnambool (C),142.590393,-38.386281
2,Government,1,Avoca Primary School,Primary,O,118 Barnett Street,Avoca,VIC,3467,03 5465 3176,599,Pyrenees (S),143.475649,-37.084502
3,Government,1,Avenel Primary School,Primary,O,40 Anderson Street,Avenel,VIC,3664,03 5796 2264,643,Strathbogie (S),145.234722,-36.901368
4,Government,1,Warrandyte Primary School,Primary,O,5-11 Forbes Street,Warrandyte,VIC,3113,03 9844 3537,421,Manningham (C),145.213980,-37.742675


In [6]:
crime_data = pd.read_excel(crime_rate_path, sheet_name = "Table 03")
crime_data.head()

,Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded
0,2021,March,Alpine,3691,Dederang,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,1
1,2021,March,Alpine,3691,Glen Creek,C Drug offences,C30 Drug use and possession,C32 Drug possession,1
2,2021,March,Alpine,3691,Gundowring,C Drug offences,C10 Drug dealing and trafficking,C12 Drug trafficking,1
3,2021,March,Alpine,3691,Gundowring,E Justice procedures offences,E10 Justice procedures,E13 Resist or hinder officer,1
4,2021,March,Alpine,3691,Kergunyah South,B Property and deception offences,B40 Theft,B49 Other theft,1


In [7]:
cleaned_crime_data = clean_crime_data(crime_data)

cleaned_crime_data.to_csv(crime_file, index = False, encoding = "utf-8")

cleaned_crime_data.head()

,Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded
0,2018,March,Alpine,3691,Dederang,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,1
1,2018,March,Alpine,3691,Dederang,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,1
2,2018,March,Alpine,3691,Dederang,A Crimes against the person,Other crimes against the person,Other crimes against the person,1
3,2018,March,Alpine,3691,Dederang,C Drug offences,C10 Drug dealing and trafficking,C12 Drug trafficking,1
4,2018,March,Alpine,3691,Glen Creek,D Public order and security offences,D10 Weapons and explosives offences,D11 Firearms offences,2
